<a href="https://colab.research.google.com/github/sergejhorvat/Tensorflow2.0_Udemy/blob/master/Distributed_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Installing dependencies and setting up the environment

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 96kB/s 
     |████████████████████████████████| 419kB 41.5MB/s 
     |████████████████████████████████| 3.0MB 30.0MB/s 


## Stage 2: Importing project dependencies

In [0]:
import time
import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'2.0.0-alpha0'

## Stage 3: Dataset preprocessing

### Loading the MNIST dataset

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Image normalization

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

In [0]:
X_train.shape

(60000, 28, 28)

### Dataset reshaping

As we are using fully connected layers, input dimension vector should be 1D for all pixels so we have to do a reshape for all data.

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Stage 4: Distributed Training

### Defining normal (non distributed) model

In [0]:
model_normal = tf.keras.models.Sequential()

In [0]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [0]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [0]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [0]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Defining the Distributed Strategy

In [14]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0


### Defining a distributed model

In [15]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

### Speed comparison between normal training and distributed training process

Google colab is not a multiGPU environment so this test is pointless in this environment. It's used to show how it's done in proper evironments.

In [16]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {}".format(time.time() - start_time))

INFO:tensorflow:batch_all_reduce invoked for batches size = 4 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
Epoch 1/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2813 - sparse_categorical_accuracy: 0.9179
Epoch 2/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.1368 - sparse_categorical_accuracy: 0.9593
Epoch 3/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.1040 - sparse_categorical_accuracy: 0.9682
Epoch 4/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.0852 - sparse_categorical_accuracy: 0.9733
Epoch 5/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.0732 - sparse_categorical_accuracy: 0.9772
Epoch 6/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.0644 - sparse_categorical_accuracy: 0.9799
Epoch 7/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.0562 - sparse_categorica

In [17]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {}".format(time.time() - start_time))

Epoch 1/10
60000/60000 [==============================] - 11s 190us/sample - loss: 0.2898 - sparse_categorical_accuracy: 0.9159
Epoch 2/10
60000/60000 [==============================] - 12s 194us/sample - loss: 0.1422 - sparse_categorical_accuracy: 0.9577
Epoch 3/10
60000/60000 [==============================] - 12s 194us/sample - loss: 0.1063 - sparse_categorical_accuracy: 0.9676
Epoch 4/10
60000/60000 [==============================] - 11s 183us/sample - loss: 0.0883 - sparse_categorical_accuracy: 0.9725
Epoch 5/10
60000/60000 [==============================] - 11s 185us/sample - loss: 0.0748 - sparse_categorical_accuracy: 0.9767
Epoch 6/10
60000/60000 [==============================] - 11s 185us/sample - loss: 0.0658 - sparse_categorical_accuracy: 0.9797
Epoch 7/10
60000/60000 [==============================] - 11s 186us/sample - loss: 0.0576 - sparse_categorical_accuracy: 0.9809
Epoch 8/10
60000/60000 [==============================] - 11s 184us/sample - loss: 0.0524 - sparse_categ